In [1]:
import pandas as ps
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import transformers

# custom classes
from TFRecordRead_Class import TFRecordRead
from mySecrets import secrets


2024-11-24 16:04:59.273948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-24 16:04:59.359556: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 16:04:59.934649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/larkinan/miniconda3/envs/tf/lib/:/home/larkinan/miniconda3/envs/tf/l

ninja: no work to do.


In [2]:
def loadDataset(dataFolder):
    recordReader = TFRecordRead(dataFolder,128,2)
    dataset = recordReader.tfRecords.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(128)
    #val_data = val_data.batch(VAL_BATCH_SIZE)
    print("loaded dataset")
    return(dataset)

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 128
#trainData = loadDataset(DATA_FOLDER)
testData = loadDataset(secrets['TEST_DATA_FOLDER'])

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/mnt/e/Aspire/BERT/child/test/strict/
1
found 1 tf records 





2024-11-24 16:05:04.419289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-24 16:05:04.446120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-24 16:05:04.446163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-24 16:05:04.454221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-24 16:05:04.454307: I tensorflow/compile

parsing
dict_keys(['input_ids', 'labels', 'masks', 'vectors'])
parsed
loaded dataset


In [4]:
def getLabels():
    labelSet = []
    for images, labels in testData.take(1000):
        if(len(labelSet)==0):
            labelSet = labels.numpy()
        else:
            labelSet = np.concatenate((labelSet,labels))
    print(labelSet.shape)
    return(labelSet)

In [5]:
vectorTextModel = tf.keras.models.load_model(secrets['TEXT_NO_VECTOR_MODEL_FILEPATH'],custom_objects={"TFBertModel": transformers.TFBertModel})
vectorTextModel.compile(loss=[tf.keras.losses.BinaryCrossentropy()],optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

Model config BertConfig {
  "_name_or_path": "/mnt/h/Aspire/BERT/testSaveBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30623
}



In [6]:
def getPredictions(dataset,model):
    labelSet,predictions = [],[]
    for text, labels in testData.take(1000):
        print(text)
        pred = model.predict(text)
        if(len(labelSet)==0):
            labelSet = labels.numpy()
            predictions = pred
        else:
            labelSet = np.concatenate((labelSet,labels))
            predictions = np.concatenate((predictions,pred))
    return([labelSet,predictions])
labels,predictions = getPredictions(testData,vectorTextModel)

(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
array([[  101,  1030, 10930, ...,     0,     0,     0],
       [  101,  1045,  1521, ...,     0,     0,     0],
       [  101,  1523,  1030, ...,     0,     0,     0],
       ...,
       [  101,  2178, 11081, ...,     0,     0,     0],
       [  101,  1001,  1042, ...,     0,     0,     0],
       [  101,  2026,  4845, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(128, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>)
1/4 [======>.......................] - ETA: 2s

2024-11-24 16:05:14.474063: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


4/4 [==============================] - 1s 63ms/step
(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
array([[ 101, 7359, 3740, ...,    0,    0,    0],
       [ 101, 1030, 2012, ...,    0,    0,    0],
       [ 101, 1030, 4138, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2514, ...,    0,    0,    0],
       [ 101, 2000, 2102, ...,    0,    0,    0],
       [ 101, 5091, 4299, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: shape=(128, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>)
4/4 [==============================] - 0s 19ms/step
(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
array([[  101,  2065,  2017, ...,     0,     0,     0],
       [  101, 20916,  7685, ...,     0,     0,     0],
       [  101,  1030,  4080, ...,     0,     0,     0],
       ...,
       [  101,  

In [7]:
print(tf.sigmoid(predictions[0:2]))
predictions2 = tf.round(tf.nn.sigmoid(predictions))
print(predictions2[0:2])
print(predictions2.shape)

tf.Tensor(
[[6.8960845e-02 9.1587715e-03 4.9864957e-03 4.8298547e-03 1.2423381e-02
  2.8530845e-01]
 [6.3206287e-05 4.8922029e-05 5.9159440e-05 3.0350011e-05 1.5316844e-04
  1.4219175e-03]], shape=(2, 6), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]], shape=(2, 6), dtype=float32)
(5000, 6)


In [8]:
def calcTruePositive(labels,predictions,debug=False):
    correct = np.multiply(labels,predictions)
    nCorrect = np.sum(correct,axis=0)
    if(debug):
        percCorrect = nCorrect/np.sum(labels,axis=0)*100
        return([nCorrect,percCorrect])
    return(nCorrect)
#calcTruePositive(labels,predictions)

In [9]:
def calcTrueNegative(labels,predictions,debug=False):
    labZeros = np.where(labels==0,1,0)
    predZeros = np.where(predictions==0,1,0)
    correct = np.multiply(labZeros,predZeros)
    nCorrect = np.sum(correct,axis=0)
    if debug:
        percCorrect = nCorrect/np.sum(labZeros,axis=0)*100
        return([nCorrect,percCorrect])
    return(nCorrect)
#calcTrueNegative(labels,predictions)

In [10]:
def calcFalseNegative(labels,predictions,debug=False):
    predZeros = np.where(predictions==0,1,0)
    correct = np.multiply(labels,predZeros)
    nCorrect = np.sum(correct,axis=0)
    if debug:
        percCorrect = nCorrect/np.sum(labels,axis=0)*100
        return([nCorrect,percCorrect])
    return(nCorrect)
#calcFalseNegative(labels,predictions)

In [11]:
def calcFalsePositive(labels,predictions,debug=False):
    labZeros = np.where(labels==0,1,0)
    correct = np.multiply(labZeros,predictions)
    nCorrect = np.sum(correct,axis=0)
    if(debug):
        percCorrect = nCorrect/np.sum(labels,axis=0)*100
        return([nCorrect,percCorrect])
    return(nCorrect)
#calcFalsePositive(labels,predictions)

In [12]:
def calcConfusionMatrix(labels,predictions):
    predictionsRound = tf.round(tf.nn.sigmoid(predictions))
    confusionMatrix = ps.DataFrame({
        'truePositive':calcTruePositive(labels,predictionsRound).astype(int),
        'trueNegative':calcTrueNegative(labels,predictionsRound).astype(int),
        'falsePositive':calcFalsePositive(labels,predictionsRound).astype(int),
        'falseNegative':calcFalseNegative(labels,predictionsRound).astype(int)
    })
    return(confusionMatrix)
calcConfusionMatrix(labels,predictions)

,truePositive,trueNegative,falsePositive,falseNegative
0,312,4545,55,88
1,163,4726,19,92
2,26,4894,7,73
3,87,4868,14,31
4,109,4793,18,80
5,1094,3434,239,233
